In [1]:
import bs4, urllib, re, pandas as pd, numpy as np

In [31]:
collection_dir = "/Users/bradleyallen/Desktop/My Notes - Burroughs Collection/"
index_filename = collection_dir + "index.html"
items = []
unicode_space_ptrn = re.compile('\xa0')
with open(index_filename, encoding='utf-8') as html:
    index_soup = bs4.BeautifulSoup(html, 'html.parser')
    for link in index_soup.find_all('a'):
        filename = collection_dir + urllib.parse.unquote(link.get('href'))
        with open(filename, encoding='utf-8') as html:
            soup = bs4.BeautifulSoup(html, 'html.parser')
            title = re.sub(unicode_space_ptrn, ' ', soup.title.string)
            if soup.div.string:
                desc = re.sub(unicode_space_ptrn, ' ', soup.div.string)
            else:
                desc = 'MISSING'
            items.append([title, desc, soup.img['src']])
collection = pd.DataFrame(items, columns=['Title', 'Description', 'Image'])

In [32]:
collection["Year"] = collection.Description.str.extract(', (\d\d\d\d)', expand=True)

In [33]:
collection["Maynard and Miles"] = collection.Description.str.extract('Maynard and Miles ([A-Z]\d+[A-Za-z]*)', expand=True)

In [34]:
collection["Schottlaender"] = collection.Description.str.extract('Schottlaender ([A-Z]\d+[A-Za-z]*)', expand=True)

In [35]:
collection = collection[['Title', 'Year', 'Maynard and Miles', 'Schottlaender', 'Description', 'Image']].sort_values(by='Year')

In [38]:
collection.reset_index(drop=True, inplace=True)

In [39]:
collection.to_csv('wsb_catalog.csv', index_label="ID")